# 🖼️ ComfyUI Colab — Image Upscaler

Batch upscale images from Google Drive using ComfyUI. Drop files in, get results out.

**Run cells top to bottom.** Code is hidden by default — expand to edit.

---
| Step | What it does |
|------|-------------|
| 1 · Mount Drive | Connects My Drive or Shared Drive |
| 2 · Install | ComfyUI + speed stack (xformers, triton, sageattention) |
| 3 · Custom Nodes | Install upscaler nodes from GitHub |
| 4 · Models | Download upscale models (Real-ESRGAN etc.) |
| 5 · Launch | Starts ComfyUI + watches input folder for new images |

---
### ▶️ How to run

1. Upload images to your Drive input folder — `<your root folder>/input/`
2. In cell 5, choose your mode:
   - `QUALITY_MODE = False` — uses **Real-ESRGAN**, fast, best for digital art & anime
   - `QUALITY_MODE = True` — uses **SeedVR2**, slower, best for real photos
3. Run **Runtime → Run all**
4. Wait for all images to process — outputs appear in `<your root folder>/output/`
5. When done, go to **Runtime → Disconnect and delete runtime** to stop billing

In [1]:
# @title 📂 1 · Mount Google Drive { display-mode: "form" }
# @markdown Mounts Google Drive (My Drive + Shared Drives). Run this first before any other cell.
from google.colab import drive
import os

print("📂 Connecting to Google Drive...")
drive.mount('/content/drive', force_remount=True)

# Verify access
if os.path.exists("/content/drive/Shareddrives"):
    print("✅ Shared Drives detected.")
elif os.path.exists("/content/drive/Shared drives"):
    print("✅ Shared Drives detected (alternate path).")
else:
    print("ℹ️  Only MyDrive detected.")

print("✅ Drive mount complete.")


📂 Connecting to Google Drive...
Mounted at /content/drive
✅ Shared Drives detected.
✅ Drive mount complete.


In [ ]:
# @title ⚙️ Install ComfyUI & Dependencies { display-mode: "form" }
# @markdown ## 2 · Install ComfyUI & Dependencies
# @markdown
# @markdown Clones ComfyUI and installs the full stack including the **speed trio**:
# @markdown - **xformers** — faster attention, lower VRAM
# @markdown - **triton** — GPU kernel acceleration
# @markdown - **sageattention** — additional attention optimization
# @markdown
# @markdown Also sets `PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True` to reduce OOM crashes.
import os
from pathlib import Path

WORKSPACE = "/content/ComfyUI"

# ── 0. Install uv ─────────────────────────────────────────────────
print("📦 Installing uv...")
!curl -LsSf https://astral.sh/uv/install.sh | sh
os.environ["PATH"] = "/root/.local/bin:" + os.environ["PATH"]
print("✅ uv ready\n")

# ── 1. Clone / update ComfyUI ─────────────────────────────────────
if not os.path.exists(WORKSPACE):
    print("📥 Cloning ComfyUI...")
    !git clone -q https://github.com/comfyanonymous/ComfyUI {WORKSPACE}
    print("✅ Cloned\n")
else:
    print("✅ ComfyUI exists, pulling updates...")
    !cd {WORKSPACE} && git pull -q
    print("✅ Updated\n")

%cd {WORKSPACE}

# ── 2. PyTorch ────────────────────────────────────────────────────
print("⚡ Installing PyTorch 2.8.0...")
!uv pip install --system torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --no-deps

# ── 3. Speed stack ────────────────────────────────────────────────
print("🚀 Installing speed stack...")
!uv pip install --system --upgrade xformers triton sageattention

# ── 4. ComfyUI requirements ───────────────────────────────────────
print("📦 Installing ComfyUI requirements...")
!uv pip install --system -r requirements.txt

# ── 5. Core dependencies ──────────────────────────────────────────
print("📚 Installing core dependencies...")
!uv pip install --system \
    accelerate einops diffusers \
    "safetensors>=0.4.2" \
    aiohttp pyyaml Pillow scipy tqdm psutil \
    "tokenizers>=0.13.3" sentencepiece soundfile \
    "kornia>=0.7.1" spandrel torchsde \
    av albumentations opencv-python \
    onnxruntime-gpu color-matcher \
    comfy_aimdo comfy-kitchen

# ── 6. Transformers / HuggingFace ────────────────────────────────
print("🤗 Installing transformers & huggingface-hub...")
!uv pip install --system \
    "transformers>=4.45.0,<5.0.0" \
    "huggingface-hub>=0.23.0" \
    hf_transfer

# ── 7. CUDA memory optimization ──────────────────────────────────
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
print("✅ CUDA memory config set\n")

# ── 8. ComfyUI Manager ────────────────────────────────────────────
manager_path = f"{WORKSPACE}/custom_nodes/ComfyUI-Manager"
if not os.path.exists(manager_path):
    print("📥 Installing ComfyUI Manager...")
    !git clone -q https://github.com/ltdrdata/ComfyUI-Manager {manager_path}
else:
    print("🔄 Updating ComfyUI Manager...")
    !cd {manager_path} && git pull -q
print("✅ ComfyUI Manager ready\n")

# ── 9. Verify ─────────────────────────────────────────────────────
import importlib.metadata as meta
print("📋 Key package versions:")
for pkg in ["torch", "xformers", "transformers", "huggingface-hub", "safetensors"]:
    try:
        print(f"  ✅ {pkg}: {meta.version(pkg)}")
    except Exception:
        print(f"  ❌ {pkg}: not found")

print("\n🎉 Installation complete! Run the next cell.")


In [ ]:
# @title 🔧 3 · Custom Nodes { display-mode: "form" }
# @markdown Add GitHub repos to `CUSTOM_NODES` below and uncomment to install.
# @markdown Re-running is safe — existing nodes are skipped automatically.
import os, shutil

# ─── CONFIGURATION ───
WORKSPACE        = "/content/ComfyUI"
CUSTOM_NODES_DIR = f"{WORKSPACE}/custom_nodes"

# Add any GitHub URL here. Format: ("Folder_Name", "URL")
CUSTOM_NODES = [
    ("rgthree-comfy","https://github.com/rgthree/rgthree-comfy"),
    ("ComfyUI-SeedVR2_VideoUpscaler","https://github.com/numz/ComfyUI-SeedVR2_VideoUpscaler"),
]

# ─── INSTALLATION LOGIC ───
print("🚀 Starting Custom Node Installation...\n")

for name, url in CUSTOM_NODES:
    path = os.path.join(CUSTOM_NODES_DIR, name)

    # 1. FIX BROKEN DOWNLOADS
    # If folder exists but isn't a git repo, it's a failed download from a previous run.
    if os.path.exists(path) and not os.path.exists(os.path.join(path, ".git")):
        print(f"⚠️ Found broken folder '{name}', cleaning up...")
        shutil.rmtree(path)

    # 2. CLONE REPO
    if not os.path.exists(path):
        print(f"📥 Cloning: {name}")
        # We use ! instead of os.system to see real-time progress and errors
        !git clone {url} {path}
    else:
        print(f"⏭️  Already exists: {name}")
        # Optional: update existing nodes
        # !cd {path} && git pull

    # 3. INSTALL REQUIREMENTS
    req_file = os.path.join(path, "requirements.txt")
    if os.path.exists(req_file):
        print(f"📦 Installing dependencies for {name}...")
        # We removed -q so you can see if a specific library fails to install
        !uv pip install --system -r {req_file}
    else:
        print(f"ℹ️  No requirements.txt for {name}")

print("\n✨ All custom nodes processed.")

In [ ]:
# @title 📥 4 · Download Models { display-mode: "form" }
# @markdown Add as many models as you want in the list below.<br>
# @markdown - For **Hugging Face** → use `"type": "hf"` + `repo_id` + optional `filename` (None = full repo).<br>
# @markdown - For **Civitai / mirrors / other** → use `"type": "url"` + direct `url` + `filename`.

import os
from huggingface_hub import hf_hub_download, snapshot_download
from tqdm import tqdm
import subprocess

# Install aria2c (required for fast URL downloads)
!apt-get update -qq && apt-get install -y -qq aria2
print("✅ aria2c installed\n")

# ────────────────────────────────────────────────
#   ↓↓↓  ADD / EDIT YOUR MODELS HERE  ↓↓↓
# ────────────────────────────────────────────────

downloads = [
    # SeedVR2 models (from AInVFX / numz repos — HF → fast hf_transfer)
    # Example: Add more HF models (single file or full repo)
    {
        "type": "hf",
        "repo_id": "numz/SeedVR2_comfyUI",
        "filename": "seedvr2_ema_7b_fp16.safetensors",                           # None = download entire repo
        "folder": "models/SEEDVR2",
    },

    # Example: Non-HF download (Civitai / mirror) → uses aria2c
    {
        "type": "url",
        "url": "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth",
        "filename": "RealESRGAN_x4plus_anime_6B.pth",
        "folder": "models/upscale_models",
    },
]

# ────────────────────────────────────────────────
#     Usually no need to change anything below
# ────────────────────────────────────────────────

base_dir = "/content/ComfyUI"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"   # Enables fast Rust-accelerated HF downloads

for item in tqdm(downloads, desc="Downloading models"):
    target_folder = os.path.join(base_dir, item["folder"].lstrip("/"))
    os.makedirs(target_folder, exist_ok=True)

    if item["type"].lower() in ["hf", "huggingface"]:
        repo = item["repo_id"]
        file = item.get("filename")
        print(f"\n📥 HF   → {repo}  →  {file or 'FULL REPO'}")

        try:
            if file is None:
                snapshot_download(
                    repo_id=repo,
                    local_dir=target_folder,
                    local_dir_use_symlinks=False,
                    resume_download=True,
                )
            else:
                hf_hub_download(
                    repo_id=repo,
                    filename=file,
                    local_dir=target_folder,
                    resume_download=True,
                )
        except Exception as e:
            print(f"  ⚠️ Error with HF download: {e}")
            print("   → Check repo_id / filename or try manual URL with type:url")

    elif item["type"].lower() == "url":
        url = item["url"]
        fname = item["filename"]
        print(f"\n📥 URL  → {url}  →  {fname}")

        cmd = [
            "aria2c", "--console-log-level=error", "-c",
            "-x", "16", "-s", "16", "-j", "8", "-k", "1M",
            url, "-d", target_folder, "-o", fname
        ]
        try:
            subprocess.run(cmd, check=True)
        except Exception as e:
            print(f"  ⚠️ aria2c failed: {e}")

    else:
        print(f"⚠️ Unknown type '{item['type']}' — skipping")

print("\n✅ All downloads finished! Check folders in /content/ComfyUI/models/")

In [ ]:
# @title 🚀 Launch ComfyUI { display-mode: "form" }
# @markdown ## 5 · Launch ComfyUI
# @markdown
# @markdown Choose launch mode and configure the batch watcher below.
# @markdown
# @markdown **Launch modes:**
# @markdown - `window` — opens ComfyUI in a new browser tab (default)
# @markdown - `iframe` — embeds ComfyUI inside the notebook cell
# @markdown - `cloudflare` — public URL via Cloudflare tunnel (share across devices)
# @markdown
# @markdown **Batch watcher** monitors an input folder and auto-runs a workflow on every dropped file.
# @markdown Set `BATCH_MODE = False` to just launch ComfyUI without the watcher.
import os, time, json, shutil, threading, socket, glob, requests, uuid, subprocess
from pathlib import Path
from datetime import datetime

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# paste this right after the imports, before CONFIGURATION
import builtins
if getattr(builtins, "_comfyui_launched", False):
    print("⚠️  Already running. Use Runtime → Restart to relaunch.")
    raise SystemExit()
builtins._comfyui_launched = True

# ╔══════════════════════════════════════════════════════════════════╗
# ║  ⚙️  CONFIGURATION                                              ║
# ╚══════════════════════════════════════════════════════════════════╝

LAUNCH_MODE  = "window"     # @param ["window", "iframe", "cloudflare"]
BATCH_MODE   = True          # @param {type:"boolean"}

# Full absolute path — examples:
# My Drive    : /content/drive/MyDrive/comfyui/image-upscale
# Shared Drive: /content/drive/Shareddrives/Figuro/image-upscale
DRIVE_BASE_PATH      = "/content/drive/Shareddrives/Figuro/image-upscale"  # @param {type:"string"}
EXTRA_OUTPUT_PATH    = ""                                       # @param {type:"string"}
QUALITY_MODE         = False                                         # @param {type:"boolean"}
WORKFLOW_FILENAME    = "workflow_api.json" if QUALITY_MODE else "fast_workflow_api.json"
INPUT_NODE_ID        = "15"                                      # @param {type:"string"}
POLL_INTERVAL        = 10                                       # @param {type:"integer"}
SUPPORTED_EXTENSIONS = "png,jpg,jpeg,webp,mp4,gif,bmp,tiff"   # @param {type:"string"}

# ╔══════════════════════════════════════════════════════════════════╗
# ║  🔍  PATH RESOLUTION (works with Shared Drives + MyDrive)      ║
# ╚══════════════════════════════════════════════════════════════════╝

from google.colab import drive as _drive
if not os.path.exists("/content/drive"):
    _drive.mount('/content/drive', force_remount=True)

def validate_path(label: str, path: str) -> str:
    path = path.strip().rstrip("/")
    if not path.startswith("/content/drive/"):
        raise ValueError(f"❌ {label} must start with /content/drive/MyDrive/... or /content/drive/Shareddrives/<name>/...\n   Got: '{path}'")
    os.makedirs(path, exist_ok=True)
    return path

def safe_makedirs(path: str) -> str:
    """Create a subdir inside an existing Drive folder. Falls back to /tmp on error."""
    try:
        os.makedirs(path, exist_ok=True)
        return path
    except OSError as e:
        fallback = path.replace("/content/drive", "/tmp/drive_mirror")
        os.makedirs(fallback, exist_ok=True)
        print(f"⚠️  Cannot create '{path}': {e}")
        print(f"   → Using local fallback: '{fallback}'")
        print(f"   → Fix: create the folder manually in Drive, then re-run.")
        return fallback

# ╔══════════════════════════════════════════════════════════════════╗
# ║  🔧  SETUP                                                      ║
# ╚══════════════════════════════════════════════════════════════════╝

COMFYUI_URL = "http://127.0.0.1:8188"
EXTENSIONS  = {e.strip().lower().lstrip('.') for e in SUPPORTED_EXTENSIONS.split(',') if e.strip()}
_FILE_FIELDS = ["image","video","audio","file","mask",
                "image_path","video_path","file_path","input","source","path"]

if BATCH_MODE:
    MAIN_FOLDER   = validate_path("DRIVE_BASE_PATH", DRIVE_BASE_PATH)
    INPUT_FOLDER  = safe_makedirs(os.path.join(MAIN_FOLDER, "input"))
    PROCESSED_DIR = safe_makedirs(os.path.join(MAIN_FOLDER, "processed"))
    OUTPUT_DIR    = safe_makedirs(os.path.join(MAIN_FOLDER, "output"))
    WORKFLOW_PATH = os.path.join(MAIN_FOLDER, WORKFLOW_FILENAME)
    EXTRA_DIR     = None
    if EXTRA_OUTPUT_PATH.strip():
        EXTRA_DIR = validate_path("EXTRA_OUTPUT_PATH", EXTRA_OUTPUT_PATH.strip())
    print(f"✅ Paths ready:")
    print(f"   Input     : {INPUT_FOLDER}")
    print(f"   Processed : {PROCESSED_DIR}")
    print(f"   Output    : {OUTPUT_DIR}")
    print(f"   Workflow  : {WORKFLOW_PATH}")
else:
    MAIN_FOLDER = WORKFLOW_PATH = INPUT_FOLDER = PROCESSED_DIR = OUTPUT_DIR = EXTRA_DIR = None

# ── Batch helpers ─────────────────────────────────────────────────

def load_workflow():
    if not os.path.exists(WORKFLOW_PATH):
        raise FileNotFoundError(f"Workflow not found: {WORKFLOW_PATH}")
    with open(WORKFLOW_PATH) as f:
        return json.load(f)

def auto_detect_field(workflow, node_id):
    if node_id not in workflow:
        raise KeyError(f"Node '{node_id}' not found. Available: {list(workflow.keys())}")
    inputs = workflow[node_id].get("inputs", {})
    for known in _FILE_FIELDS:
        if known in inputs:
            return known
    media_exts = {"png","jpg","jpeg","webp","gif","bmp","tiff","tif",
                  "mp4","avi","mov","mkv","webm","mp3","wav","flac","ogg"}
    for field, val in inputs.items():
        if isinstance(val, str) and Path(val).suffix.lower().lstrip('.') in media_exts:
            return field
    short = [f for f, v in inputs.items()
             if isinstance(v, str) and len(v) < 256 and not v.startswith(("http", "{"))]
    if short:
        return short[0]
    raise ValueError(f"Can't detect input field on node '{node_id}'. Inputs: {list(inputs.keys())}")

def wait_for_comfyui(timeout=120):
    print("⏳ Waiting for ComfyUI...", end="", flush=True)
    deadline = time.time() + timeout
    while time.time() < deadline:
        try:
            if requests.get(f"{COMFYUI_URL}/system_stats", timeout=3).status_code == 200:
                print(" ✅ Ready!")
                return
        except Exception:
            pass
        print(".", end="", flush=True)
        time.sleep(2)
    raise TimeoutError("ComfyUI did not start in time.")

def queue_prompt(workflow):
    client_id = str(uuid.uuid4())
    r = requests.post(f"{COMFYUI_URL}/prompt",
                      json={"prompt": workflow, "client_id": client_id}, timeout=30)
    r.raise_for_status()
    data = r.json()
    if "error" in data:
        raise RuntimeError(f"Prompt error: {data['error']}")
    return data["prompt_id"]

def poll_until_done(prompt_id, timeout=600):
    deadline = time.time() + timeout
    while time.time() < deadline:
        time.sleep(3)
        try:
            r = requests.get(f"{COMFYUI_URL}/history/{prompt_id}", timeout=10)
            r.raise_for_status()
            history = r.json()
        except Exception:
            continue
        if prompt_id not in history:
            continue
        outputs = []
        for _, node_out in history[prompt_id].get("outputs", {}).items():
            for _, items in node_out.items():
                if isinstance(items, list):
                    for item in items:
                        if isinstance(item, dict) and "filename" in item:
                            outputs.append(item)
        return outputs
    raise TimeoutError(f"Prompt {prompt_id} timed out after {timeout}s")

def fetch_and_save_output(file_info, dest_dirs, stem):
    params = {"filename": file_info["filename"],
              "subfolder": file_info.get("subfolder", ""),
              "type": file_info.get("type", "output")}
    r = requests.get(f"{COMFYUI_URL}/view", params=params, timeout=120)
    r.raise_for_status()
    ext  = Path(file_info["filename"]).suffix
    ts   = datetime.now().strftime("%Y%m%d_%H%M%S")
    name = f"{stem}_{ts}{ext}"
    saved = []
    for dest in dest_dirs:
        os.makedirs(dest, exist_ok=True)
        out_path = os.path.join(dest, name)
        with open(out_path, 'wb') as f:
            f.write(r.content)
        saved.append(out_path)
    return saved

def copy_input_to_comfyui(src_path):
    comfyui_input = "/content/ComfyUI/input"
    os.makedirs(comfyui_input, exist_ok=True)
    filename = Path(src_path).name
    shutil.copy2(src_path, os.path.join(comfyui_input, filename))
    return filename

def process_file(filepath):
    stem = Path(filepath).stem
    print(f"\n{'─'*55}")
    print(f"📂 {Path(filepath).name}  [{datetime.now().strftime('%H:%M:%S')}]")
    workflow = load_workflow()
    field    = auto_detect_field(workflow, INPUT_NODE_ID)
    filename = copy_input_to_comfyui(filepath)
    workflow[INPUT_NODE_ID]["inputs"][field] = filename
    print(f"   Node [{INPUT_NODE_ID}].{field} → '{filename}'")
    prompt_id = queue_prompt(workflow)
    print(f"   Queued: {prompt_id}")
    print("   ⏳ Running...", end="", flush=True)
    output_files = poll_until_done(prompt_id)
    print(f" done — {len(output_files)} output(s)")
    dest_dirs = [OUTPUT_DIR] + ([EXTRA_DIR] if EXTRA_DIR else [])
    for fi in output_files:
        for p in fetch_and_save_output(fi, dest_dirs, stem):
            print(f"   💾 {p}")
    if not output_files:
        print("   ⚠️  No outputs — check workflow has a Save node.")
    dest_proc = os.path.join(PROCESSED_DIR, Path(filepath).name)
    if os.path.exists(dest_proc):
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        dest_proc = os.path.join(PROCESSED_DIR, f"{stem}_{ts}{Path(filepath).suffix}")
    shutil.move(filepath, dest_proc)
    print(f"   📦 Moved to processed")

def run_watcher():
    wait_for_comfyui()
    try:
        wf    = load_workflow()
        field = auto_detect_field(wf, INPUT_NODE_ID)
        cls   = wf[INPUT_NODE_ID].get("class_type", "unknown")
        print(f"\n✅ Workflow: {WORKFLOW_PATH}")
        print(f"   Node [{INPUT_NODE_ID}] {cls} → field: '{field}'")
    except Exception as e:
        print(f"\n⚠️  Workflow check failed: {e}")
        return
    print(f"\n👀 Watching '{INPUT_FOLDER}' every {POLL_INTERVAL}s")
    print("   Drop files into the input folder to trigger the workflow.\n")
    seen_errors = {}
    while True:
        try:
            candidates = sorted(set(
                f for ext in EXTENSIONS
                for f in glob.glob(os.path.join(INPUT_FOLDER, f"*.{ext}")) +
                          glob.glob(os.path.join(INPUT_FOLDER, f"*.{ext.upper()}"))
            ))
            for fp in candidates:
                if seen_errors.get(fp, 0) >= 3:
                    continue
                try:
                    process_file(fp)
                    seen_errors.pop(fp, None)
                except Exception as e:
                    seen_errors[fp] = seen_errors.get(fp, 0) + 1
                    print(f"\n❌ Error ({seen_errors[fp]}/3) — {Path(fp).name}: {e}")
                    if seen_errors[fp] >= 3:
                        print(f"   ⛔ Giving up on {Path(fp).name}")
        except Exception as e:
            print(f"⚠️  Watcher error: {e}")
        time.sleep(POLL_INTERVAL)

# ── Launch modes ──────────────────────────────────────────────────

def start_window(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        if sock.connect_ex(('127.0.0.1', port)) == 0:
            break
        sock.close()
    from google.colab import output as co
    print("\n🌐 ComfyUI is ready!")
    co.serve_kernel_port_as_window(port)

def start_iframe(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        if sock.connect_ex(('127.0.0.1', port)) == 0:
            break
        sock.close()
    from google.colab import output as co
    print("\n🌐 ComfyUI is ready!")
    co.serve_kernel_port_as_iframe(port, height=900)
    co.serve_kernel_port_as_window(port)

def start_cloudflare(port):
    subprocess.run(["wget", "-q",
        "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb"])
    subprocess.run(["dpkg", "-i", "cloudflared-linux-amd64.deb"], capture_output=True)
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        if sock.connect_ex(('127.0.0.1', port)) == 0:
            break
        sock.close()
    print("\n🌐 Launching Cloudflare tunnel...")
    p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
                         stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com" in l:
            print("ComfyUI URL:", l[l.find("http"):], end='')

# ── Print summary & start ─────────────────────────────────────────
print("=" * 55)
print("🗂️  ComfyUI Batch Workflow Runner")
print("=" * 55)
print(f"  Mode       : {LAUNCH_MODE} | Batch: {BATCH_MODE}")
if BATCH_MODE:
    print(f"  Input      : {INPUT_FOLDER}")
    print(f"  Output     : {OUTPUT_DIR}")
    print(f"  Workflow   : {WORKFLOW_PATH}")
    print(f"  Node ID    : {INPUT_NODE_ID}")
    print(f"  Poll       : {POLL_INTERVAL}s")
    if EXTRA_DIR:
        print(f"  Extra out  : {EXTRA_DIR}")
print("=" * 55)

%cd /content/ComfyUI

if LAUNCH_MODE == "iframe":
    threading.Thread(target=start_iframe,     daemon=True, args=(8188,)).start()
elif LAUNCH_MODE == "cloudflare":
    threading.Thread(target=start_cloudflare, daemon=True, args=(8188,)).start()
else:
    threading.Thread(target=start_window,     daemon=True, args=(8188,)).start()

if BATCH_MODE:
    threading.Thread(target=run_watcher, daemon=True).start()

print("\n🚀 Starting ComfyUI...\n")
!python main.py --highvram --cuda-malloc --use-sage-attention --fast --dont-print-server


---

## 📝 Notes

**Workflow format** — Export via ComfyUI → Settings → Dev Mode → *Save (API Format)*. Uses numeric node IDs as top-level keys.

**Input node ID** — open `workflow.json`, find your Load Image / Load Video node, its top-level key (e.g. `"12"`) is `INPUT_NODE_ID`. The input field is auto-detected and printed on startup.

**Batch mode off** — set `BATCH_MODE = False` to just launch ComfyUI without the watcher. Useful when you want to use the UI manually.

**Launch modes** — `window` opens a new tab, `iframe` embeds in the cell, `cloudflare` gives a public shareable URL.

**Error handling** — files that fail 3 times are skipped permanently so the watcher never loops.

**Live edits** — workflow JSON is re-read on every file so you can tweak it without restarting.

---
